In [50]:
import time
import urllib.parse
import pandas as pd
import requests
import json
import random
import string
import warnings
import sqlite3

# Suppress all warnings
warnings.filterwarnings("ignore")

In [51]:
# Path to the .db file
db_path = 'db/ir.db'

public_key_address = "/root/cert.crt"
private_key_address = "/root/private.key"

server_address = "host.kooft.fuck:1234"
cookie = "lang=en-US; session=MTcwNzQ1NjY4M3x...."

In [52]:
def generate_random_string(length=8):
    characters = string.ascii_lowercase + string.digits
    random_string = ''.join(random.choice(characters) for _ in range(length))
    return random_string


def create_vmess_body(row):
    body = {
        "up": row['up'],
        "down": row['down'],
        "total": row['total'],
        "remark": row['remark'],
        "enable": "true",
        "expiryTime": row['expiry_time'],
        "listen": "",
        "port": row['port'],
        "protocol": "vmess",
        "settings": json.dumps({
            "clients": [
                {
                    "id": row['uuid'],
                    "email": generate_random_string(8),  # Add email if available
                    "limitIp": 0,
                    "totalGB": 0,
                    "expiryTime": 0,
                    "enable": True,
                    "tgId": "",
                    "subId": "",  # Add subId if available
                    "reset": 0
                }
            ]
        }),
        "streamSettings": json.dumps({
            "network": "tcp",
            "security": "tls",
            "tlsSettings": {
                "serverName": row['serverName'],
                "minVersion": "1.0",
                "maxVersion": "1.3",
                "certificates": [
                    {
                        "certificateFile": public_key_address,
                        "keyFile": private_key_address,
                        "ocspStapling": 3600
                    }
                ]
            },
            "tcpSettings": {
                "acceptProxyProtocol": False,
                "header": {
                    "type": "none"
                }
            }
        }),
        "sniffing": json.dumps({
            "enabled": True,
            "destOverride": ["http", "tls", "quic", "fakedns"]
        })
    }
    return urllib.parse.urlencode(body)


def create_vless_body(row):
    """Create the body for vless protocol."""
    body = {
        "up": row['up'],
        "down": row['down'],
        "total": row['total'],
        "remark": row['remark'],
        "enable": "true",
        "expiryTime": row['expiry_time'],
        "listen": "",
        "port": row['port'],
        "protocol": "vless",
        "settings": json.dumps({
            "clients": [
                {
                    "id": row['uuid'],
                    "flow": "",
                    "email": generate_random_string(8),  # Add email if available
                    "limitIp": 0,
                    "totalGB": 0,
                    "expiryTime": 0,
                    "enable": True,
                    "tgId": "",
                    "subId": "",  # Add subId if available
                    "reset": 0
                }
            ],
            "decryption": "none",
            "fallbacks": []
        }),
        "streamSettings": json.dumps({
            "network": "tcp",
            "security": "tls",
            "tlsSettings": {
                "serverName": row['serverName'],
                "minVersion": "1.0",
                "maxVersion": "1.3",
                "certificates": [
                    {
                        "certificateFile": public_key_address,
                        "keyFile": private_key_address,
                        "ocspStapling": 3600
                    }
                ]
            },
            "tcpSettings": {
                "acceptProxyProtocol": False,
                "header": {
                    "type": "none"
                }
            }
        }),
        "sniffing": json.dumps({
            "enabled": True,
            "destOverride": ["http", "tls", "quic", "fakedns"]
        })
    }
    return urllib.parse.urlencode(body)

In [53]:

# Establish a connection to the SQLite database
conn = sqlite3.connect(db_path)

# The SQL query to select all data from the desired table
# Replace 'your_table_name' with the actual name of the table you want to query
query = "SELECT * FROM inbounds"

# Read the data into a pandas DataFrame
df = pd.read_sql_query(query, conn)

# Close the connection
conn.close()

def extract_id(settings_str):
    try:
        settings_dict = json.loads(settings_str)
        # Assuming 'clients' is always a list and you want the 'id' of the first client
        client_id = settings_dict['clients'][0]['id']
        return client_id
    except (KeyError, IndexError, json.JSONDecodeError):
        # In case of any error, return None or some default value
        return None

# Apply the function to the 'settings' column to create a new 'client_id' column
df['uuid'] = df['settings'].apply(extract_id)

# Function to extract 'serverName', 'network', and 'security' from the 'stream_settings' column
def extract_details(stream_settings_str):
    try:
        # Load the string as JSON
        settings_json = json.loads(stream_settings_str)
        # Access the necessary items
        server_name = settings_json['tlsSettings']['serverName']
        network = settings_json['network']
        security = settings_json['security']
        return server_name, network, security
    except (KeyError, json.JSONDecodeError):
        # In case JSON is malformed or keys don't exist, return None or placeholders
        return None, None, None

# Apply the function to the 'stream_settings' column
df[['serverName', 'network', 'security']] = df.apply(lambda row: extract_details(row['stream_settings']), axis=1, result_type="expand")

# Function to update 'certificateFile' and 'keyFile' in the 'stream_settings' column
def update_cert_and_key(stream_settings_str):
    try:
        # Load the string as JSON
        settings_json = json.loads(stream_settings_str)
        # Update the 'certificateFile' and 'keyFile' paths
        if settings_json.get('tlsSettings', {}).get('certificates'):
            for cert in settings_json['tlsSettings']['certificates']:
                cert['certificateFile'] = public_key_address
                cert['keyFile'] = private_key_address
        # Convert JSON back to string
        updated_stream_settings_str = json.dumps(settings_json)
        return updated_stream_settings_str
    except json.JSONDecodeError:
        # In case JSON is malformed, return the original string
        return stream_settings_str

# Apply the function to the 'stream_settings' column to update the values
df['stream_settings'] = df['stream_settings'].apply(update_cert_and_key)

print(df['stream_settings'].iloc[0])
# df.sample()

{"network": "tcp", "security": "tls", "tlsSettings": {"serverName": "ir55.yaqoot.top", "certificates": [{"certificateFile": "/root/cert55.crt", "keyFile": "/root/private55.key"}]}, "tcpSettings": {"header": {"type": "none"}}}


In [54]:
# Define the API endpoint and headers
api_url = f"https://{server_address}/panel/inbound/add"
headers = {
    "accept": "application/json, text/plain, */*",
    "accept-language": "en-US,en;q=0.9,fa;q=0.8,zh-CN;q=0.7,zh;q=0.6",
    "content-type": "application/x-www-form-urlencoded; charset=UTF-8",
    "sec-ch-ua": "\"Not_A Brand\";v=\"8\", \"Chromium\";v=\"120\", \"Google Chrome\";v=\"120\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "\"Windows\"",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-origin",
    "x-requested-with": "XMLHttpRequest",
    "Cookie": cookie
}

# List to track inbounds for which no API has been sent
no_api_sent = []

# Update the iteration loop to use these functions
for index, row in df.iterrows():
    print(f"We are on row: {row['remark']}")
    if row['network'] == 'tcp' and row['security'] == 'tls' and row['protocol'] != 'trojan':
        if row['protocol'] == 'vmess':
            body = create_vmess_body(row)
        elif row['protocol'] == 'vless':
            body = create_vless_body(row)
        else:
            print(f"{row['remark']} Ignored")
            no_api_sent.append(row['remark'])
            continue
        time.sleep(1)
        try:
            response = requests.post(api_url, headers=headers, data=body)
            print("Response:", response.status_code, response.text)
        except Exception as e:
            print(f"Error sending API Request for '{row['remark']}' - Protocol: {row['protocol']}: {e}")

    else:
        print(f"{row['remark']} Ignored")
        no_api_sent.append(row['remark'])

# Print the inbounds for which no API has been sent
print("\nNo API has been sent for the following inbounds:")
print(no_api_sent)

We are on row: Samir55
Response: 200 {"success":true,"msg":"Create Successful","obj":{"id":41,"up":1789896877,"down":42636329972,"total":0,"remark":"Samir55","enable":true,"expiryTime":0,"clientStats":null,"listen":"","port":47624,"protocol":"vmess","settings":"{\"clients\": [{\"id\": \"fc6b01f5-0ee0-42b3-aeae-659c916d9977\", \"email\": \"6jrwpxq5\", \"limitIp\": 0, \"totalGB\": 0, \"expiryTime\": 0, \"enable\": true, \"tgId\": \"\", \"subId\": \"\", \"reset\": 0}]}","streamSettings":"{\"network\": \"tcp\", \"security\": \"tls\", \"tlsSettings\": {\"serverName\": \"ir55.yaqoot.top\", \"minVersion\": \"1.0\", \"maxVersion\": \"1.3\", \"certificates\": [{\"certificateFile\": \"/root/cert55.crt\", \"keyFile\": \"/root/private55.key\", \"ocspStapling\": 3600}]}, \"tcpSettings\": {\"acceptProxyProtocol\": false, \"header\": {\"type\": \"none\"}}}","tag":"inbound-0.0.0.0:47624","sniffing":"{\"enabled\": true, \"destOverride\": [\"http\", \"tls\", \"quic\", \"fakedns\"]}"}}
We are on row: Naz